<a href="https://colab.research.google.com/github/alinealinealine/LangChain_Experiments/blob/main/HackAIThon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Career Coach project

## Prepare the environment

In [ ]:
!python --version

Python 3.10.11


In [ ]:
!pip install openai
!pip install langchain
!pip install chromadb
!pip install tiktoken
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import openai
import os
import langchain

In [ ]:
#Entering API Credentials
os.environ["OPENAI_API_KEY"] = "./openai_api.txt"
openai.api_key_path = "./openai_api.txt"

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

## Where you are now


Input description of your career goal and provide your resume in PDF.

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader('./resume_sample.txt')

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

In [ ]:
query = "Can you summarize what is the skills does the job candidate have? Please organize it in bulle points. \
Give me three typical profile for people who has similar experience with this job candidate. \
Organize the output in HTML format"
response = index.query(query)

from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Alternative using txt to test.

In [ ]:
resume = f"""
Sarah Johnson

Email: sarah.johnson@email.com
Phone: (555) 123-4567
LinkedIn: linkedin.com/in/sarahjohnson
Address: 123 Elm Street, City, ST, 98765

Objective:
Marketing professional with 8 years of experience in the technology industry, seeking to leverage skills in team management, marketing strategy, and market trend analysis to transition to a career focused on environmental sustainability.

Professional Experience:

1. ABC Tech Solutions: Marketing Manager (January 2014 - Present)

- Manage a team of 10 marketing specialists in planning and executing campaigns for a leading technology company, contributing to a 50% increase in annual revenue
- Develop integrated marketing strategies across digital and traditional channels, driving brand awareness and revenue growth
- Analyze market trends, customer data, and competitor performance to identify opportunities for new product launches and target marketing efforts
- Collaborate with the product development, sales, and engineering teams to ensure marketing alignment with the overall company objectives
- Oversee content creation for the company blog, social media, and email marketing campaigns, leading to a 30% increase in website traffic and user engagement
- Successfully launched four new products, resulting in a 25% increase in market share within their respective segments

2. XYZ Digital Studio: Assistant Marketing Manager (June 2010 - December 2013)

- Assisted the Marketing Manager in planning and implementing campaigns, resulting in a 20% increase in product sales and customer acquisition during the tenure
- Conducted market research to understand customer needs, preferences, and buying behavior, informing product development and marketing approaches
- Managed social media, PPC campaigns, and SEO optimization, driving brand visibility and customer engagement
- Coordinated and managed trade shows, conferences, and product launch events to establish a strong brand presence within the industry
- Developed and maintained relationships with key partners and influencers, leading to vital collaborations that bolstered brand perception

Education:

Bachelor of Business Administration, Major in Marketing (2006 - 2010)
XYZ State University

Certifications and Skills:

- Certified Professional Marketer (CPM) from the American Marketing Association
- Google Analytics and Google Ads Certifications
- Proficient in marketing tools such as HubSpot, Salesforce, and MailChimp
- Excellent written and verbal communication skills
- Strong analytical and problem-solving abilities
- Familiarity with market research techniques and tools
- Knowledge of environmental sustainability topics and trends

Volunteer Experience:

- Volunteer Marketing Coordinator, Green City Initiative (2016 - Present)
Develop and manage marketing campaigns for a local nonprofit organization focused on promoting sustainable living practices, including social media management and email newsletters

Interests:

Environmental conservation, renewable energy, sustainable living, hiking, gardening

Memberships:

- American Marketing Association (AMA)
- Environmental Professionals Network (EPN)
"""

## What you want to do

Option 1: input your own goal

In [ ]:
career_goal = f"""
Sarah has been working as a marketing manager \
for the past eight years in a technology company. \
However, she has recently developed a strong interest in sustainable development \
and wants to transition her career into the field of environmental conservation. \
She has gained experience in managing teams, developing marketing strategies, \
and analyzing market trends. Sarah is now seeking guidance on how to leverage her skills \
and experience to make a successful transition into a career focused on environmental sustainability.
"""

Option 2: explore by chating with bot

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [ ]:
import panel as pn  # GUI
pn.extension()
panels = [] # collect display 

add_content = "This is additional content."
context = [ {'role':'system', 'content':"""
You are a professional career coach for mid-career planing, you first greet the candidate, then collects the information about the candidate. \
Your goal is to help the candidate to discover his career interest and goal, step by step.
Please provide customized service for the job market candidate based on his resume:
"""} ]  # accumulate messages
context.append({'role': 'system', 'content': resume})

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

## How to get there

### Prompt

In [ ]:
prompt = f"""
The career goal of the job market candidate is: ```{career_goal}```
Based on the resume: ```{resume}```
Please provide information below step by step (please generate the output as a report in HTML format): 

1. An overview of the industry insight, the job market for the recommended jobs. (keep it around 500 words)
2. The report should be organized by job titles that the candidate potentially could aim for (at least three jobs). 
3. Under each job title:
   a. Provide a job description. 
   b. Matching score - give a score of evaluating the matching level at a salce of 10 (make it in bold and in color)
   c. Strength and Weakness - explain the reasoning of the score, lay out strength and weakness. (make the output display as a SWOT analysis)
4. Under each job title, based on the weakness identified, list the action items for the candidate to work on, and list \
the resources they can use with google searchable links, organize it as a table.
5. Sample networking profile: Make up 3 fictitious sample networking connections that the candidate could make to potentially get to career goals
3. Under each job title, list the companies the candidate can work for, organize the information as a table, include company information below:
    - Company Name: the name of the company.
    - Location: the location of the company.
    - Industry: the industry the company belongs to.
    - Relevance: explain why this company is recommended.  

"""

In [ ]:
response = get_completion(prompt)
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Strengths,Weaknesses
8 years of experience in marketing Experience managing a team of marketing specialists Developed and implemented integrated marketing strategies Strong analytical and problem-solving abilities,Limited experience in environmental conservation Needs to develop a stronger understanding of sustainable living practices and products
Strengths,Weaknesses
8 years of experience in marketing Experience conducting market research Strong analytical and problem-solving abilities Knowledge of environmental sustainability topics and trends,Limited experience in environmental conservation Needs to develop a stronger understanding of sustainable living practices and products
Strengths,Weaknesses
8 years of experience in marketing Experience managing a team of marketing specialists Developed and implemented integrated marketing strategies Knowledge of environmental sustainability topics and trends,Limited experience in environmental conservation Needs to develop a stronger understanding of sustainable living practices and products Limited experience in developing and maintaining relationships with key partners and influencers
Action Items,Resources
Research environmental conservation organizations and initiatives Attend conferences and events focused on environmental sustainability Volunteer with local environmental conservation organizations,The Nature Conservancy GreenBiz Events VolunteerMatch
Action Items,Resources
Research environmental conservation organizations and initiatives Attend conferences and events focused on environmental sustainability Volunteer with local environmental conservation organizations,The Nature Conservancy GreenBiz Events VolunteerMatch
Action Items,Resources


### Agent

In [ ]:
!pip install google-search-results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached google_search_results-2.4.2-py3-none-any.whl


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain import LLMMathChain

In [ ]:
os.environ["SERPAPI_API_KEY"] = "3e94670070f9962595995ae7e315bdc4184ea6fb93b8b7e2a27f790419380882"

In [ ]:
search = SerpAPIWrapper()
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

In [ ]:
model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executer=executor, verbose=True)

In [ ]:
agent.run("I want to enter climate tech industry, can you recommend me network connections?")

## Application Assistant

Job description

In [ ]:
prompt = f"""
Generate a typical job description for Carbon Capture and Storage Analyst in climate tech industry.
"""
jd = get_completion(prompt)

Writeup assistant

In [ ]:
prompt = f"""
The resume of the job market candidate is: ```{resume}```
Based on the job description: ```{jd}```

Please generate the content below in HTML format.
1. Refine the resume. 
2. Write a one page cover letter. 
"""

In [ ]:
resume_new = get_completion(prompt)
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(resume_new))

Job search plan

In [ ]:
time = f"""
3 month
"""
prompt = f"""
The job the candidate wants to apply for: ```{jd}````
The job candidate's resume: ```{resume}```.

The candidate want to find the job in ```{time}```. By evaluating the gap bteween the job description and resume, \
please develop a plan to help the candidate prepare for the job application. The output should be in Latex format, \
and organized as a gantte chart in a calendar tracking progress, with milestones highlighted, list the detailed tasks.
"""

plan = get_completion(prompt)
from IPython.display import display, Markdown, Latex, HTML, JSON
display(Latex(plan))

Simulate the interview quesiton and answer:
Maybe use bing for answers?

In [ ]:
prompt = f"""
The job the candidate wants to apply for: ```{jd}````
The job candidate's resume: ```{resume}```.

Please simulate the interview quesetions and answer to help the job candidate prepare the interview for this job. \
Please note the answers should follow the Amazon principles? 
The output should be in html format. The questions and answers should be organized as pairs in a table.
"""

interview = get_completion(prompt)
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(interview))

Question,Answer
Can you tell us about your experience with carbon capture and storage analysis?,"As a policy analyst, I have conducted research and data analysis on a range of public policy issues, including climate change. I have also collaborated with cross-functional teams to develop and implement policies aimed at reducing carbon emissions. While I do not have direct experience with carbon capture and storage analysis, I am confident in my ability to learn quickly and apply my analytical skills to this role."
How do you stay up-to-date with the latest developments in carbon capture and storage technologies?,I regularly read industry publications and attend conferences and webinars to stay informed about the latest developments in carbon capture and storage technologies. I also collaborate with colleagues and industry experts to share knowledge and insights.
Can you give an example of a time when you had to collaborate with cross-functional teams to implement a policy?,"As a senior policy analyst, I collaborated with the economic development team and the city planning department to develop and implement a policy aimed at incentivizing the development of green infrastructure. We worked together to gather data and insights, develop policy recommendations, and present our findings to policymakers and stakeholders. The policy was successfully implemented and resulted in increased investment in green infrastructure projects."
How do you approach problem-solving?,"I approach problem-solving by first gathering all relevant data and insights. I then analyze the data to identify patterns and trends, and develop hypotheses about potential solutions. I collaborate with colleagues and stakeholders to test these hypotheses and refine our approach. I also prioritize transparency and communication throughout the problem-solving process to ensure that all stakeholders are informed and engaged."
Can you give an example of a time when you had to present complex data in a clear and engaging manner?,"As a policy analyst, I presented findings and recommendations to policymakers, stakeholders, and the general public on a regular basis. One example was when I presented data on the economic impact of a proposed policy aimed at reducing carbon emissions. I used data visualization tools to present the data in an easily understandable format, and I prioritized clear and engaging communication to ensure that all stakeholders understood the potential impact of the policy."
